# SHAP Analysis for Model Explainability

## Goal
- Compute SHAP values for the best model.
- Visualize feature importance.
- Explain individual predictions.

In [ ]:
import shap
import joblib
import pandas as pd
import hopsworks

# Load Model and Scaler
project = hopsworks.login()
mr = project.get_model_registry()
model_meta = mr.get_best_model('aqi_forecaster', 'rmse', 'min')
model_dir = model_meta.download()

model = joblib.load(f'{model_dir}/model.pkl')
scaler = joblib.load(f'{model_dir}/scaler.pkl')

# Load Data
fs = project.get_feature_store()
aqi_fg = fs.get_feature_group('aqi_readings', version=1)
df = aqi_fg.read()
# Preprocess (same as training)
X = df.drop(columns=['date', 'aqi']).dropna()
X_scaled = scaler.transform(X)

# Compute SHAP values
explainer = shap.Explainer(model, X_scaled)
shap_values = explainer(X_scaled)

shap.summary_plot(shap_values, X)